## Notebook to obtain the density of pharmacies in New York City by borough

In [133]:
import pandas as pd

tables = pd.read_html("https://www.op.nysed.gov/professions/pharmacist/license-statistics#stores")
df = tables[2]

In [134]:
df

,County,Number,County.1,Number.1,County.2,Number.2
0,Albany,73,Jefferson,22,Saratoga,41.0
1,Allegany,10,Kings,884,Schenectady,32.0
2,Bronx,477,Lewis,4,Schoharie,6.0
3,Broome,50,Livingston,12,Schuyler,4.0
4,Cattaraugus,18,Madison,11,Seneca,5.0
5,Cayuga,16,Monroe,139,Steuben,20.0
6,Chautauqua,28,Montgomery,13,St. Lawrence,29.0
7,Chemung,20,Nassau,345,Suffolk,301.0
8,Chenango,10,New York,640,Sullivan,17.0
9,Clinton,18,Niagara,51,Tioga,5.0


In [135]:
pop_tables = pd.read_html("https://www.newyork-demographics.com/counties_by_population")

pop_df = pop_tables[0]


In [136]:
pop_df.tail()

,Rank,County,Population
58,59,Lewis County,26681
59,60,Yates County,24808
60,61,Schuyler County,17920
61,62,Hamilton County,5068
62,United States Census Bureau. B01001 SEX BY AGE...,United States Census Bureau. B01001 SEX BY AGE...,United States Census Bureau. B01001 SEX BY AGE...


In [137]:
# remove the last row

pop_df = pop_df[:-1]

In [138]:
pop_df

,Rank,County,Population
0,1,Kings County,2712360
1,2,Queens County,2393104
2,3,New York County,1669127
3,4,Suffolk County,1522998
4,5,Bronx County,1468262
...,...,...,...
57,58,Schoharie County,29936
58,59,Lewis County,26681
59,60,Yates County,24808
60,61,Schuyler County,17920


In [139]:
# take only the rows for the 5 boroughs of NYC

pop_df = pop_df[pop_df['County'].isin(['New York County', 'Kings County', 'Queens County', 'Bronx County', 'Richmond County'])]


In [140]:
pop_df

,Rank,County,Population
0,1,Kings County,2712360
1,2,Queens County,2393104
2,3,New York County,1669127
4,5,Bronx County,1468262
9,10,Richmond County,493194


In [141]:
pharmacies_dict = {
    'New York County': 640,
    'Kings County': 884,
    'Queens County': 846,
    'Bronx County': 477,
    'Richmond County': 103
}

In [142]:
# read the dictionary into a dataframe

pharmacies_df = pd.DataFrame.from_dict(pharmacies_dict, orient='index', columns=['Pharmacies'])

In [143]:
pharmacies_df

,Pharmacies
New York County,640
Kings County,884
Queens County,846
Bronx County,477
Richmond County,103


In [144]:
# join the two dataframes

pharmacies_df = pharmacies_df.join(pop_df.set_index('County'))

In [145]:
pharmacies_df

,Pharmacies,Rank,Population
New York County,640,3,1669127
Kings County,884,1,2712360
Queens County,846,2,2393104
Bronx County,477,5,1468262
Richmond County,103,10,493194


In [146]:
pharmacies_df['Pharmacies'] = pharmacies_df['Pharmacies'].astype(int)
pharmacies_df['Population'] = pharmacies_df['Population'].astype(int)

In [147]:
# calculate the number of pharmacies per 10,000 people

pharmacies_df['Pharmacies per 10,000'] = pharmacies_df['Pharmacies'] / pharmacies_df['Population'] * 10000

In [148]:
# round the result to 2 decimal places

pharmacies_df['Pharmacies per 10,000'] = pharmacies_df['Pharmacies per 10,000'].round(2)

In [149]:
pharmacies_df

,Pharmacies,Rank,Population,"Pharmacies per 10,000"
New York County,640,3,1669127,3.83
Kings County,884,1,2712360,3.26
Queens County,846,2,2393104,3.54
Bronx County,477,5,1468262,3.25
Richmond County,103,10,493194,2.09


In [150]:
# mean pharmacies per 10,000 people

pharmacies_df['Pharmacies per 10,000'].mean()

3.194

In [151]:
boroughs_dict = {
    'New York County': 'Manhattan',
    'Kings County': 'Brooklyn',
    'Queens County': 'Queens',
    'Bronx County': 'Bronx',
    'Richmond County': 'Staten Island'
}

# join boroughs to pharmacies dataframe

pharmacies_df = pharmacies_df.join(pd.DataFrame.from_dict(boroughs_dict, orient='index', columns=['Borough']))

In [152]:
pharmacies_df

,Pharmacies,Rank,Population,"Pharmacies per 10,000",Borough
New York County,640,3,1669127,3.83,Manhattan
Kings County,884,1,2712360,3.26,Brooklyn
Queens County,846,2,2393104,3.54,Queens
Bronx County,477,5,1468262,3.25,Bronx
Richmond County,103,10,493194,2.09,Staten Island


In [153]:
# remove index column

pharmacies_df = pharmacies_df.reset_index()

In [154]:
pharmacies_df

,index,Pharmacies,Rank,Population,"Pharmacies per 10,000",Borough
0,New York County,640,3,1669127,3.83,Manhattan
1,Kings County,884,1,2712360,3.26,Brooklyn
2,Queens County,846,2,2393104,3.54,Queens
3,Bronx County,477,5,1468262,3.25,Bronx
4,Richmond County,103,10,493194,2.09,Staten Island


In [155]:
pharmacies_df = pharmacies_df.rename(columns={'index': 'County'})

In [156]:
# save the dataframe to a csv file

pharmacies_df.to_csv('nyc_pharmacy_density.csv', index=False)
